In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import os

os.makedirs('/content/object_detection_demo_flow/fine_tuned_model/', exist_ok=True)
os.makedirs('/content/object_detection_demo_flow/training/', exist_ok=True)

!cp -r "/content/gdrive/My Drive/Capstone/Model/20.11.01_sota/fine_tuned_model/" "/content/object_detection_demo_flow/"
!cp -r "/content/gdrive/My Drive/Capstone/Model/20.11.01_sota/training/" "/content/object_detection_demo_flow/"

In [3]:
%%capture
import re
import numpy as np

output_directory = '/content/object_detection_demo_flow/fine_tuned_model'
model_dir = '/content/object_detection_demo_flow/training'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [4]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [5]:
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)
!ls -alh {pb_fname}

-rw------- 1 root root 19M Nov 16 15:37 /content/object_detection_demo_flow/fine_tuned_model/frozen_inference_graph.pb


In [6]:
os.makedirs('/content/object_detection_demo_flow/data/images/final_test', exist_ok=True)
!cp -r "/content/gdrive/My Drive/Capstone/Sample/" "/content/object_detection_demo_flow/data/images/final_test"

In [7]:

os.makedirs('/content/object_detection_demo_flow/data/annotations/', exist_ok=True)
!cp -r "/content/gdrive/My Drive/Capstone/Model/20.11.01_sota/annotations/" "/content/object_detection_demo_flow/data/annotations"

label_map_pbtxt_fname = '/content/object_detection_demo_flow/data/annotations/annotations/label_map.pbtxt'

In [8]:
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR = '/content/object_detection_demo_flow/data/images/final_test/Sample'

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

['/content/object_detection_demo_flow/data/images/final_test/Sample/20201004_184116.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201004_174303.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201005_165613.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201005_165446.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201004_184102.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201005_183029.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201004_172307.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201005_165239.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201005_162234.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201004_184137.jpg', '/content/object_detection_demo_flow/data/images/final_test/Sample/20201004_174421.jpg', '/content/object_det

In [9]:

%tensorflow_version 1.x
!pip install tf_slim
#!pip install gast==0.2.2

TensorFlow 1.x selected.
     |████████████████████████████████| 358kB 5.7MB/s 


In [11]:
%cd /content/gdrive/My Drive/Capstone/Model/models/research/object_detection



import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

bounding_box_location = []
crop_images = []

from collections import defaultdict
from io import StringIO
# This is needed to display the images.
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

num_classes = get_num_classes(label_map_pbtxt_fname)

print(num_classes)


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    print(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    image_np, bounding_box_location = vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)
    plt.show()
    plt.clf()
    plt.close('all')
    
    if(len(bounding_box_location) != 0):
      print("##################crop######################")
      crop_img = image.crop((bounding_box_location[0], bounding_box_location[2], bounding_box_location[1], bounding_box_location[3]))
      crop_images.append(crop_img)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(crop_img)
      plt.show()
      del bounding_box_location[:]
      plt.clf()
      plt.close('all')

Output hidden; open in https://colab.research.google.com to view.